In [1]:
import numpy as np
import modelskill as ms

In [2]:
fn = '../tests/testdata/SW/HKZN_local_2017_DutchCoast.dfsu'
mr = ms.ModelResult(fn, name='HKZN_local', item=0)
mr.data

Dfsu2D
number of elements: 958
number of nodes: 570
projection: LONG/LAT
number of items: 15
time: 23 steps with dt=10800.0s
      2017-10-27 00:00:00 -- 2017-10-29 18:00:00

Configuration of comparison, see [SW_DutchCoast.ipynb](SW_DutchCoast.ipynb) for more details.

In [3]:
o1 = ms.PointObservation('../tests/testdata/SW/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA")
o2 = ms.PointObservation("../tests/testdata/SW/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL")
o3 = ms.TrackObservation("../tests/testdata/SW/Alti_c2_Dutch.dfs0", item=3, name="c2")
cc = ms.compare([o1, o2, o3], mr)
cc

<ComparerCollection>
Comparer: HKNA
Comparer: EPL
Comparer: c2

Standard set of metrics

In [4]:
cc.skill().style(precision=2)

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
EPL,67,-0.07,0.22,0.21,0.19,0.97,0.08,0.93
HKNA,386,-0.19,0.35,0.29,0.25,0.97,0.09,0.91
c2,113,-0.00,0.35,0.35,0.29,0.97,0.13,0.90


Select a specific metric

In [5]:
cc.skill(metrics="mean_absolute_error")

,n,mean_absolute_error
observation,,
EPL,67,0.188513
HKNA,386,0.251839
c2,113,0.294585


Some metrics has parameters, which require a bit special treatment.

In [6]:
from modelskill.metrics import hit_ratio

def hit_ratio_05_pct(obs, model):
    return hit_ratio(obs, model, 0.5) * 100

def hit_ratio_01_pct(obs, model):
    return hit_ratio(obs, model, 0.1) * 100

cc.skill(metrics=[hit_ratio_05_pct, hit_ratio_01_pct]).style(precision=0)

,n,hit_ratio_05_pct,hit_ratio_01_pct
observation,,,
EPL,67,99,27
HKNA,386,87,30
c2,113,86,17


It is also possible to do it in a single line using a lambda function (anonymous function), with the downside that the anonymous function has no name, and thus no automatic column label :-(

In [7]:
cc.skill(metrics=lambda obs, model : hit_ratio(obs, model, 0.1))

,n,<lambda>
observation,,
EPL,67,0.268657
HKNA,386,0.295337
c2,113,0.168142


And you are of course always free to specify your own special metric.

In [8]:
def my_special_metric(obs, model):

    res = obs - model

    res_clipped = np.clip(res,0,np.inf)

    return np.mean(np.abs(res_clipped))


cc.skill(metrics=my_special_metric)

,n,my_special_metric
observation,,
EPL,67,0.127555
HKNA,386,0.223049
c2,113,0.147897
